## Installing Dependencies

In [1]:
! pip install pandas
! pip install openpyxl


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

## Load Excel Data

In [3]:
csod_file_path = 'csod_schema.xlsx'
test_data_file_path = 'Test.xlsx'

csod_data = pd.ExcelFile(r'C:\Users\Briankechy\source\data\DataAutomation\csod_schema.xlsx')
test_data = pd.ExcelFile(r'C:\Users\Briankechy\source\data\DataAutomation\Test.xlsx')


## Load Excel Files

In [4]:
# File paths
csod_file_path = r'C:\Users\Briankechy\source\data\DataAutomation\csod_schema.xlsx'
test_data_file_path = r'C:\Users\Briankechy\source\data\DataAutomation\Test.xlsx'

# Load Excel files
csod_data = pd.ExcelFile(csod_file_path)
test_data = pd.ExcelFile(test_data_file_path)

## Load Data from Each retailer

In [5]:
def load_data(excel_file):
    carrefour_df = pd.read_excel(excel_file, sheet_name="CARREFOUR")
    naivas_df = pd.read_excel(excel_file, sheet_name="NAIVAS")
    quickmatt_df = pd.read_excel(excel_file, sheet_name="QUICKMATT")
    return carrefour_df, naivas_df, quickmatt_df


## Standardize columns

In [6]:
# Transform: Standardize column names
def standardize_columns(carrefour_df, naivas_df, quickmatt_df):
    # Standardize Carrefour columns
    carrefour_df = carrefour_df.rename(columns={
        'Dept Name': 'Department',
        'Section Name': 'Section',
        'Family name': 'Family',
        'Sub Family Name': 'SubFamily',
        'Brand Name': 'Brand',
        'SKU Description': 'Itemname'
    })
    carrefour_df['Retailer'] = 'CARREFOUR'

    # Standardize Naivas columns
    naivas_df = naivas_df.rename(columns={
        'Storename': 'Store',
        'Subcategory': 'SubFamily',
        'Suppliername': 'Supplier',
        'Netamountincl': 'Total_Sales'
    })
    naivas_df['Retailer'] = 'NAIVAS'

    # Standardize Quickmatt columns
    quickmatt_df = quickmatt_df.rename(columns={
        'STORE_NAME': 'Store',
        'ITEM_CODE': 'ItemCode',
        'ITEM_NAME': 'Itemname',
        'DEPARTMENT': 'Department',
        'SUB DEPARTMENT': 'SubDepartment',
        'MICRO DEPARTMENT': 'MicroDepartment',
        'TOTAL SALES': 'Total_Sales'
    })
    quickmatt_df['Retailer'] = 'QUICKMATT'

    return carrefour_df, naivas_df, quickmatt_df

##  Match products and generate unified Product ID

In [7]:
# Transform: Match products and generate unified Product ID
def match_products_and_generate_id(carrefour_df, naivas_df, quickmatt_df):
    # Select relevant columns from each dataframe
    carrefour_products = carrefour_df[['Itemname', 'Brand', 'Family', 'SubFamily', 'Retailer']]
    naivas_products = naivas_df[['Itemname', 'Brand', 'Subcategory', 'Retailer']]
    quickmatt_products = quickmatt_df[['Itemname', 'ItemCode', 'Department', 'SubDepartment', 'MicroDepartment', 'Retailer']]

    # Concatenate all products
    all_products = pd.concat([carrefour_products, naivas_products, quickmatt_products], ignore_index=True)

    # Remove duplicates based on Itemname and Retailer
    all_products = all_products.drop_duplicates(subset=['Itemname', 'Retailer'])

    # Generate Product ID
    all_products['Product_ID'] = [f'PROD_{i:05d}' for i in range(1, len(all_products) + 1)]

    return all_products


## master Dataset

In [8]:
# Load: Create master dataset
def create_master_dataset(all_products, carrefour_df, naivas_df, quickmatt_df):
    # Merge the Product_ID back to original dataframes
    carrefour_merged = pd.merge(carrefour_df, all_products[['Itemname', 'Product_ID', 'Retailer']], 
                                on=['Itemname', 'Retailer'], how='left')
    naivas_merged = pd.merge(naivas_df, all_products[['Itemname', 'Product_ID', 'Retailer']], 
                             on=['Itemname', 'Retailer'], how='left')
    quickmatt_merged = pd.merge(quickmatt_df, all_products[['Itemname', 'Product_ID', 'Retailer']], 
                                on=['Itemname', 'Retailer'], how='left')

    # Combine all data into a master dataset
    master_dataset = pd.concat([carrefour_merged, naivas_merged, quickmatt_merged], ignore_index=True)

    return master_dataset

main

run only this for poc

In [8]:
import pandas as pd

# File paths
test_data_file_path = r'C:\Users\Briankechy\source\data\DataAutomation\Test.xlsx'

# Load Excel file
test_data = pd.ExcelFile(test_data_file_path)

# Load ITEMMASTER (the reference)
itemmaster_df = pd.read_excel(test_data, sheet_name='ITEMMASTER')

# Function to process each supermarket and update ITEMMASTER sequentially
def process_supermarket_data(supermarket_df, itemmaster_df, unique_identifier, desc_column, barcode_column=None, brand_column=None):
    # Prepare list to collect new entries
    new_entries = []

    # Iterate through each row in the supermarket data
    for idx, row in supermarket_df.iterrows():
        unique_id = row.get(unique_identifier)
        description = row.get(desc_column)
        barcode = row.get(barcode_column) if barcode_column else None
        brand = row.get(brand_column) if brand_column else None

        # Check if unique identifier or barcode exists in ITEMMASTER
        if unique_id not in itemmaster_df['prodcode'].values:
            if barcode is None or barcode not in itemmaster_df['Barcode(WITH GEN)'].values:
                # Add the new entry
                new_entry = {
                    'prodcode': unique_id,
                    'DESC': description,
                    'Barcode(WITH GEN)': barcode,
                    'Brand': brand
                }
                new_entries.append(new_entry)

    # Convert new entries to DataFrame and concatenate with ITEMMASTER
    if new_entries:
        new_entries_df = pd.DataFrame(new_entries)
        itemmaster_df = pd.concat([itemmaster_df, new_entries_df], ignore_index=True)

    return itemmaster_df

# Process each supermarket sequentially and update ITEMMASTER
naivas_df = pd.read_excel(test_data, sheet_name='Naivas')
itemmaster_df = process_supermarket_data(naivas_df, itemmaster_df, unique_identifier='Itemid', desc_column='Itemname')

quickmatt_df = pd.read_excel(test_data, sheet_name='Quickmatt')
itemmaster_df = process_supermarket_data(quickmatt_df, itemmaster_df, unique_identifier='ITEM_CODE', desc_column='ITEM_NAME', barcode_column='BARCODE')

carrefour_df = pd.read_excel(test_data, sheet_name='Carrefour', skiprows=6)
itemmaster_df = process_supermarket_data(carrefour_df, itemmaster_df, unique_identifier='Item Code', desc_column='Item Name', barcode_column='Item Bar Code')

magunas_df = pd.read_excel(test_data, sheet_name='Magunas')
for idx, row in magunas_df.iterrows():
    sku_description = row.get('SKU-DESCRIPTION')
    if pd.notna(sku_description):
        item_code, *description_parts = sku_description.split('-')
        item_code = item_code.strip()
        description = '-'.join(description_parts).strip()

        if item_code not in itemmaster_df['prodcode'].values:
            new_entry = {
                'prodcode': item_code,
                'DESC': description,
                'Barcode(WITH GEN)': None,
                'Brand': None
            }
            new_entries = [new_entry]
            new_entries_df = pd.DataFrame(new_entries)
            itemmaster_df = pd.concat([itemmaster_df, new_entries_df], ignore_index=True)

chandarana_df = pd.read_excel(test_data, sheet_name='Chandarana ', skiprows=1)
for idx, row in chandarana_df.iterrows():
    item_name = row.get('Item Name')
    barcode = row.get('Barcode')

    if barcode not in itemmaster_df['Barcode(WITH GEN)'].values:
        new_entry = {
            'prodcode': None,
            'DESC': item_name,
            'Barcode(WITH GEN)': barcode,
            'Brand': None
        }
        new_entries = [new_entry]
        new_entries_df = pd.DataFrame(new_entries)
        itemmaster_df = pd.concat([itemmaster_df, new_entries_df], ignore_index=True)

# Function to remove duplicates based on key identifiers
def remove_duplicates(itemmaster_df):
    # Drop duplicates based on prodcode and Barcode(WITH GEN), keeping the first occurrence
    itemmaster_df.drop_duplicates(subset=['prodcode', 'Barcode(WITH GEN)'], keep='first', inplace=True)
    return itemmaster_df

# Function to complete additional fields by cross-referencing with supermarket data
def complete_missing_fields(itemmaster_df, supermarket_dfs):
    for df in supermarket_dfs:
        for idx, row in df.iterrows():
            unique_id = row.get('Itemid') or row.get('ITEM_CODE') or row.get('Item Code') or row.get('SKU') or row.get('Barcode')
            if pd.notna(unique_id):
                # Locate the row in ITEMMASTER
                mask = (itemmaster_df['prodcode'] == unique_id) | (itemmaster_df['Barcode(WITH GEN)'] == row.get('Barcode'))
                # Update Brand and other fields if they are missing in ITEMMASTER but present in the current row
                if mask.any():
                    itemmaster_idx = itemmaster_df.index[mask][0]
                    if pd.isna(itemmaster_df.at[itemmaster_idx, 'Brand']) and row.get('Brand'):
                        itemmaster_df.at[itemmaster_idx, 'Brand'] = row.get('Brand')
    return itemmaster_df

# Sequentially remove duplicates
itemmaster_df = remove_duplicates(itemmaster_df)

# Complete missing fields by cross-referencing with the supermarket data
supermarket_dfs = [naivas_df, quickmatt_df, carrefour_df, magunas_df, chandarana_df]
itemmaster_df = complete_missing_fields(itemmaster_df, supermarket_dfs)

# Save updated ITEMMASTER to a new Excel file
itemmaster_df.to_csv('updated_itemmaster_final_with_cleanup.csv', index=False)
print("Updated ITEMMASTER saved to 'updated_itemmaster_final_with_cleanup.csv'")


Updated ITEMMASTER saved to 'updated_itemmaster_final_with_cleanup.csv'


In [1]:
import pandas as pd

# Load the workbook
file_path = r'C:\Users\Briankechy\source\data\DataAutomation\Test.xlsx'
excel_file = pd.ExcelFile(file_path)

# Loop through each sheet and save it as a separate Excel file
for sheet_name in excel_file.sheet_names:
    # Read each sheet into a DataFrame
    df = excel_file.parse(sheet_name)
    
    # Define the output file name (uses the sheet name for clarity)
    output_file = f'{sheet_name}.xlsx'
    
    # Save the DataFrame to a new Excel file
    df.to_excel(output_file, index=False)
    print(f"Saved {output_file}")


Saved Synopsis.xlsx
Saved ITEMMASTER.xlsx
Saved Naivas.xlsx
Saved Quickmatt.xlsx
Saved Carrefour.xlsx
Saved Magunas.xlsx
Saved Chandarana .xlsx


## Setting Up the Database, and SQL statements

In [ ]:
table_name = 'test_table'

def generate_sql_statements(df, column_mapping, table_name):
    insert_statement = []
    for_, row in df.iterrows():
        #create a list of mapped Values
        values = [
            row['year'],
            row['TOTAL QNTY'],
            row['TOTAL VALUE']
        ]
        
        #Format Values for SQL compatibility
    formatted_values = [f"'{value}'" if isinstance(value, str) else value for value in values]
    
    sql_statement = f"INSERT INTO {table_name} (Code, U_ProductDes, U_StoreName) VALUES ({formatted_values[0]}, {formatted_values[1]}, {formatted_values[2]});"
    instert_statement.append(sql_statement)
    
    return insert_statement

OUTPUT SQL STATEMENTS TO A .SQL FILE

In [ ]:
with open('insert.sql', 'w') as f:
    for statement in generate_sql_statements:
        f.write(f"{statement}\n")
print ("SQL insert statements have been written to insert_statement.sql")